In [1]:
# third-party
import pandas as pd
import scipy
import pickle

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_breath_parameters_performance
from delay_performance import normality_test
from performance import bland_altman_plot, get_relative_errors


In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

### Activity-specific only RP analysis

In [3]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [4]:
performance_all_participants = get_breath_parameters_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,SNB,MAG,0.31 $\pm$ 0.47,14.24 $\pm$ 19.64,0.30 $\pm$ 0.43,11.16 $\pm$ 14.15,0.38 $\pm$ 0.61,7.90 $\pm$ 11.65
1,SNB,PZT,0.55 $\pm$ 0.58,27.41 $\pm$ 24.80,0.57 $\pm$ 0.59,21.62 $\pm$ 17.02,0.45 $\pm$ 0.56,9.73 $\pm$ 11.04
2,SGB,MAG,1.08 $\pm$ 0.87,25.23 $\pm$ 22.32,1.14 $\pm$ 0.84,20.77 $\pm$ 18.37,0.76 $\pm$ 0.74,7.25 $\pm$ 7.24
3,SGB,PZT,1.39 $\pm$ 1.08,32.23 $\pm$ 22.87,1.64 $\pm$ 1.26,29.33 $\pm$ 23.37,0.89 $\pm$ 1.11,8.70 $\pm$ 11.03
4,MIXB,MAG,0.35 $\pm$ 0.44,14.33 $\pm$ 16.50,0.34 $\pm$ 0.46,11.50 $\pm$ 13.60,0.34 $\pm$ 0.43,6.17 $\pm$ 7.13
5,MIXB,PZT,0.77 $\pm$ 0.68,30.56 $\pm$ 24.48,0.83 $\pm$ 0.96,25.66 $\pm$ 21.36,0.67 $\pm$ 0.83,11.67 $\pm$ 12.86
6,STNB,MAG,0.30 $\pm$ 0.47,12.79 $\pm$ 17.04,0.32 $\pm$ 0.48,11.87 $\pm$ 16.90,0.32 $\pm$ 0.55,6.50 $\pm$ 10.75
7,STNB,PZT,0.58 $\pm$ 0.53,26.70 $\pm$ 21.47,0.58 $\pm$ 0.54,22.19 $\pm$ 16.15,0.39 $\pm$ 0.51,8.23 $\pm$ 9.59
8,MCH,MAG,0.39 $\pm$ 0.37,20.68 $\pm$ 16.54,0.36 $\pm$ 0.34,19.13 $\pm$ 17.53,0.33 $\pm$ 0.38,8.57 $\pm$ 9.04
9,MCH,PZT,0.64 $\pm$ 0.54,31.52 $\pm$ 23.47,0.58 $\pm$ 0.51,29.99 $\pm$ 23.29,0.60 $\pm$ 0.58,15.26 $\pm$ 14.84


### Overall RP analysis

In [5]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [6]:
get_breath_parameters_performance(overview_all, target=None)

N=3407 tI for MAG
N=3357 tI for PZT
N=3391 tE for MAG
N=3345 tE for PZT
N=3278 tB for MAG
N=3233 tB for PZT


,Parameter,Sensor,Abs. error (s),Rel. error (%),Slope,Intercept,R^2
0,tI,MAG,0.39 $\pm$ 0.43,19.40 $\pm$ 18.79,1.03,0.00,0.74
1,tI,PZT,0.62 $\pm$ 0.57,31.27 $\pm$ 24.39,0.93,0.14,0.54
2,tE,MAG,0.39 $\pm$ 0.43,18.20 $\pm$ 17.70,0.88,0.19,0.75
3,tE,PZT,0.64 $\pm$ 0.61,29.56 $\pm$ 23.81,0.98,0.01,0.62
4,tB,MAG,0.35 $\pm$ 0.46,8.26 $\pm$ 10.13,0.99,0.02,0.90
5,tB,PZT,0.55 $\pm$ 0.63,13.20 $\pm$ 14.43,0.98,0.05,0.83


In [7]:
import plotly.graph_objects as go

for metric in ["tI", "tE", "tB"]:
    rel_error_mag = get_relative_errors(overview_all["MAG"][f"{metric} airflow (s)"], overview_all["MAG"][f"{metric} (s)"])
    rel_error_pzt = get_relative_errors(overview_all["PZT"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"])

    print(f"{metric}: {scipy.stats.mannwhitneyu(rel_error_mag, rel_error_pzt)}")

    fig = go.Figure()
    fig.add_trace(go.Violin(y=rel_error_mag, name="MAG"))
    fig.add_trace(go.Violin(y=rel_error_pzt, name="PZT"))

    # fig.show()

    # normality_test(rel_error_mag, sensor="MAG", type=f"relative error {metric}")
    # normality_test(rel_error_pzt, sensor="PZT", type=f"relative error {metric}")


tI: MannwhitneyuResult(statistic=5493990.0, pvalue=0.0051449487993406404)
tE: MannwhitneyuResult(statistic=5574095.5, pvalue=0.2224879189949569)
tB: MannwhitneyuResult(statistic=5277637.0, pvalue=0.7793203307817962)


In [8]:
import numpy as np
import plotly.graph_objects as go
from constants import CATEGORICAL_PALETTE

rel_error_all = get_relative_errors(overview_all["MAG"]["tB airflow (s)"], overview_all["MAG"]["tB (s)"])

fig = go.Figure()
categorical_palette = iter(CATEGORICAL_PALETTE)
fig.add_trace(go.Violin(y=rel_error_all,
                            #line_color=next(categorical_palette), 
                            name="overall"))

for activity in overview_all_participants.keys():

    overview_all_except_activity = transform_overview_on_overall(overview, ignore_key=activity)

    rel_error_all_except = get_relative_errors(overview_all_except_activity["MAG"]["tB airflow (s)"], overview_all_except_activity["MAG"]["tB (s)"])
    rel_error_by_activity_mag = get_relative_errors(overview_all_participants[activity]["MAG"]["tB airflow (s)"], overview_all_participants[activity]["MAG"]["tB (s)"])
    rel_error_by_activity_pzt = get_relative_errors(overview_all_participants[activity]["PZT"]["tB airflow (s)"], overview_all_participants[activity]["PZT"]["tB (s)"])
    # print(f"{activity}: {scipy.stats.ttest_ind(rel_error_all, rel_error_by_activity)}")
    print(f"{activity}: {scipy.stats.mannwhitneyu(rel_error_all_except, rel_error_by_activity_pzt)}")
    #normality_test(rel_error_by_activity, sensor="MAG", type="relative error by activity")
    
    # fig.add_trace(go.Violin(y=rel_error_by_activity,
    #                         #line_color=next(categorical_palette), 
    #                         name=activity))
#fig.show()


SNB: MannwhitneyuResult(statistic=311293.0, pvalue=0.7785786640969998)
SGB: MannwhitneyuResult(statistic=128148.5, pvalue=0.8484930368787709)
MIXB: MannwhitneyuResult(statistic=245318.5, pvalue=0.39693834744286305)
STNB: MannwhitneyuResult(statistic=321714.5, pvalue=0.6579438481280546)
MCH: MannwhitneyuResult(statistic=340527.5, pvalue=0.6256733959449536)
SQT: MannwhitneyuResult(statistic=361196.5, pvalue=0.2822937892894778)
AAL: MannwhitneyuResult(statistic=378425.0, pvalue=0.8097607061700464)
AAR: MannwhitneyuResult(statistic=396231.5, pvalue=0.2244863199950451)
ALL: MannwhitneyuResult(statistic=393581.5, pvalue=0.8367504293132895)
ALR: MannwhitneyuResult(statistic=350833.0, pvalue=0.8431904003157136)
UAL: MannwhitneyuResult(statistic=338975.5, pvalue=0.6971122532165699)
UAR: MannwhitneyuResult(statistic=345080.0, pvalue=0.862179988344383)
SE: MannwhitneyuResult(statistic=364519.0, pvalue=0.8377651357098654)
SS: MannwhitneyuResult(statistic=311881.0, pvalue=0.15384153066752482)
TR: M

In [9]:
overview_all_activities = transform_overview_on_target(overview, target="ID")
overview_all_activities.keys()

fig = go.Figure()
categorical_palette = iter(CATEGORICAL_PALETTE)
fig.add_trace(go.Violin(y=rel_error_all,
                            #line_color=next(categorical_palette), 
                            name="overall"))

# print(f"Overall: {np.mean(rel_error_all)*100:.2f} +/- {np.std(rel_error_all)*100:.2f}")

for id in overview_all_activities.keys():

    overview_all_except_id= transform_overview_on_overall(overview, ignore_key=id)

    rel_error_by_participant = get_relative_errors(overview_all_activities[id]["MAG"]["tB airflow (s)"], overview_all_activities[id]["MAG"]["tB (s)"])
    #print(f"{id}: {scipy.stats.ttest_ind(rel_error_all, rel_error_by_participant)}")
    #normality_test(rel_error_by_participant, sensor="MAG", type="relative error by id")
    fig.add_trace(go.Violin(y=rel_error_by_participant,
                            name=id))
# fig.show()
    

### Bias and variability analysis by activity

In [10]:
from performance import get_bias_variability

get_bias_variability(overview_all_participants, target="Activity", metric="tI", relative_error=False)

,Activity,Sensor,Bias (s),Variability (s)
0,SNB,MAG,0.116,0.557
1,SNB,PZT,0.270,0.758
2,SGB,MAG,0.378,1.341
3,SGB,PZT,-0.588,1.670
4,MIXB,MAG,0.181,0.529
5,MIXB,PZT,0.231,1.007
6,STNB,MAG,0.071,0.554
7,STNB,PZT,0.325,0.713
8,MCH,MAG,-0.010,0.541
9,MCH,PZT,-0.064,0.831


In [11]:
for metric in ["tI", "tE", "tB"]:
        bland_altman_plot(overview_all["MAG"][f"{metric} (s)"], overview_all["MAG"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"], overview_all["PZT"][f"{metric} airflow (s)"], metric)


MAG | mean: 0.057, lloa: -1.07, uloa: 1.18 
PZT | mean: 0.008, lloa: -1.65, uloa: 1.66 


MAG | mean: -0.065, lloa: -1.19, uloa: 1.06 
PZT | mean: -0.029, lloa: -1.77, uloa: 1.71 


MAG | mean: -0.007, lloa: -1.14, uloa: 1.13 
PZT | mean: -0.014, lloa: -1.64, uloa: 1.62 
